In [15]:
:set -XOverloadedStrings

import Control.Monad (unless)
import Control.Monad.Trans.State.Strict as SS
import Data.Attoparsec.ByteString
import Data.Byteable
import qualified Data.ByteString.Lazy   as L
import qualified Data.ByteString        as B
import qualified Codec.Compression.Zlib as Z
import Pipes
import Prelude hiding (take)
import qualified Pipes.ByteString as PB
import qualified Pipes.Prelude as PP
import qualified Pipes.Zlib as PZ
import qualified Pipes.Attoparsec as PA
import qualified System.IO as SI

import Duffer.Pack.Parser
import Duffer.Pack.Entries
import Duffer.Loose.Parser

In [2]:
compressedContent = L.toStrict $ Z.compress "compressed content"
uncompressedContent = L.toStrict $ "uncompressed content"
mingled = B.append compressedContent uncompressedContent
mingledProducer = yield mingled
decompressed = PZ.decompress' PZ.defaultWindowBits mingledProducer
(Right (value, restProducer)) <- next decompressed
value
(Left (Left finalProducer)) <- next restProducer
(Right (final, _)) <- next finalProducer
final

"compressed content"

"uncompressed content"

In [5]:
parsePackfileHeader = (word8s $ B.unpack "PACK") *> (take 4) *> (fromBytes <$> take 4)

parsePackFile path = do
    stream <- PB.fromHandle <$> SI.openFile path SI.ReadMode
    return (yield stream)

packfileContent <- B.readFile ".git/objects/pack/pack-018ead6d1f7cc653f4fa50651e92995203187170.pack"
either error id $ parseOnly parsePackfileHeader packfileContent

1735

In [22]:
unpackNextEntry = do
    Just (Right typeLen) <- PA.parse parseTypeLen
    remainder <- get
    PB.drawByte
    Just levelByte <- PB.peekByte
    let level = getCompressionLevel levelByte
    let decompressed = PZ.decompress' PZ.defaultWindowBits remainder
    return (decompressed, uncurry encodeTypeLen typeLen, level)

handle <- SI.openFile ".git/objects/pack/pack-018ead6d1f7cc653f4fa50651e92995203187170.pack" SI.ReadMode
parsePackfile = do
    Just (Right (lenHeader, noOfObjects)) <- PA.parseL parsePackfileHeader
    unpackNextEntry
    
result <- runStateT parsePackfile (PB.fromHandle handle)
(dec, header, level) = fst result
(Right (value, restProducer)) <- next dec
value
level
either error id $ parseOnly parseTypeLen header

"tree c6d1e6926b38d24a7d96500cf768d99db3463b85\nparent 16f306775ca4f748ded5ecdb8cf2c51ffbd95af6\nauthor Vaibhav Sagar <vaibhavsagar@gmail.com> 1473110396 -0400\ncommitter Vaibhav Sagar <vaibhavsagar@gmail.com> 1473110396 -0400\n\nRename single packed object resolution function.\n"

DefaultCompression

(CommitObject,273)